# FlockMTL Extension Tutorial

### You work for an e-commerce company that sells consumer electronics. You have a set of customer reviews, and your goal is to:

#### 1. Analyze the sentiment of each review, and get the top K similar reviews
#### 2. Identify high-impact reviews that require urgent attention.
#### 3. Extract recurring themes or topics to understand common issues and areas for product improvement.
#### 4. Track customer satisfaction trends over time.

### Let us see how to build SQL queries that use `FlockMTL` to achieve all of this.


   
#### Step 1: Load your OpenAI API key into the current env
* We are loading it via the .env file using dotenv python pkg.
* Create a file called .env in the current working directory, with the following text:
* `export OPENAI_API_KEY='<your OPENAI API KEY'`
* Replace the placeholder with your API key.


You can also set key as an environment variable, using `export` or `set`, depending upon your terminal


In [ ]:
!pip install python-dotenv
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

#### Step 2: Install and import DuckDB

We are using DuckDB version 1.1.1 for this tutorial

In [ ]:
#!pip install duckdb==1.1.1
import duckdb
print(duckdb.__version__)


If everything goes well, you should see the version being printed, its 1.1.1 for our example

#### Step 3: Create a database for our use

1. We create a new persistent database connection for our example called `mydb.db`.\
2. We also pass a config dict `config={'allow_unsigned_extensions' : 'true'}` to allow us to load the extension.
3. We also load a CSV file called `product_reviews.csv` into a table called `product_reviews`

In [ ]:
# Create a connection to an in-memory database
# You can also provide a persistent database by replacing `:memory` with your database name
con = duckdb.connect(':memory:', config={'allow_unsigned_extensions' : 'true'})

csv_path = 'product_reviews.csv' #add your own path
con.execute(f"CREATE TABLE product_reviews AS SELECT * FROM read_csv_auto('{csv_path}')");


#### Step 4: Load our extension


We use the DuckDB `INSTALL` and `LOAD` commands to load our extension\
Set the `path` variable to the location where the extension binary is downloaded, and then execute the following


In [ ]:
#path = '<FlockMTL extension binary path>' # replace your path here
con.execute(f"LOAD '{path}'")


#### The FlockMTL extension is now loaded into DuckDB
* Out of the box we provide support for two different LLMs - `gpt-4o` and `gpt-4o-mini`, the default being `gpt-4o-mini`
* You can list the available models by runnning the  query `GET MODELS;`

In [ ]:
con.execute("GET MODELS;").fetchall()


* We also provide a default `hello-world` prompt
* You can list the available running by runnning the query `GET PROMPTS;` 

In [ ]:
con.execute("GET PROMPTS;").fetchall()


#### Let us run the hello-world prompt

Execute the following query to run the `hello-world` prompt. We use `default` as our LLM, which is `gpt-4o-mini`

In [ ]:
query = """
SELECT llm_complete('hello-world', 'default');
"""

result = con.execute(query).fetchall()

for row in result:
    print(row)

### And that's it ! Our extension is loaded, now we can use LLM capabilities within SQL!

Using models, prompts, and llm scalar functions directly with SQL, we enable semantic analysis into our system



### Let's go through the tasks one by one

#### Performing Sentiment Analysis on Product Reviews:
* The first step is to analyze the sentiment of each review. We can use LLMs to generate a detailed sentiment analysis by combining the review text with the star rating. We also extract top 5 similar reviews for each review

* There are two steps in this task - first defining a `sentiment-analysis` prompt, and second, using the prompt to perform sentiment analysis on the table

* We use `llm_complete_json`, a function that calls the `default` model to perform sentiment analysis on each review.
`default` is an alias for `gpt-4o-mini` 

* The model evaluates the sentiment based on both the review text and the star rating.

* This step gives a sentiment score or label (e.g., positive, negative, neutral) for each review, providing a solid foundation for deeper analysis.


In [ ]:

# First we create a new sentiment analysis prompt
sentiment_analysis_prompt = """
Analyze the sentiment of the following product review. Consider both the review text and the star rating. Provide a brief sentiment label (positive, negative, or neutral) and a short explanation for your decision.

Review: {review}
Star Rating: {rating}

Output your response in the following JSON format:
{
    "sentiment": "positive/negative/neutral",
    "explanation": "Brief explanation of the sentiment analysis"
}
"""

# Use an f-string to insert the prompt directly into the query
sentiment_analysis_prompt_query = f"""
    CREATE PROMPT ('sentiment-analysis', '{sentiment_analysis_prompt}');
"""

con.execute (sentiment_analysis_prompt_query)



# Now we use the new prompt for analytical analysis
query = """
CREATE TABLE sentiment_analysis AS
WITH sentiment_analysis AS (
    SELECT 
        ProductID as product_id, 
        ID AS review_id,
        Review AS review_text, 
        Rating AS star_rating, 
        llm_complete_json('sentiment-analysis', 'default', {'review': review_text, 'rating': star_rating}) AS sentiment_json
    FROM 
        product_reviews
)
SELECT * 
FROM sentiment_analysis;
"""

# Run the query and fetch results
results = con.execute(query).fetchall()

for row in results:
    print(row)

#### Lets take a look at the newly created sentiment_analysis table

In [ ]:
query = "select * from sentiment_analysis;"
results = con.execute(query).fetchall()

for row in results:
    print(row)

The `sentiment-analysis` prompt is now available to be reused directly for later tasks as well, no need for redefinition

#### Getting top k similar reviews
* We want the top k similar reviews for each review
* We define a new prompt `get-embedding` to generate embeddings for each review, using the `llm_complete` function
* We use the embeddings to perform `cosine similarity` for each pair of reviews
* We then return `top-k` similar review to each review

In [ ]:
# Define the get-embedding prompt
get_embedding_prompt = """
    Generate an embedding vector for the following product review. 
    This embedding should capture the key semantic meaning of the review text.

    Review: {review}

    Output the embedding as FLOAT ARRAY datatype of size 3, do not return as JSON and VARCHAR.
"""

# Create the SQL query to register the get-embedding prompt
get_embedding_prompt_query = f"""
    CREATE PROMPT ('get-embedding', '{get_embedding_prompt}');
"""

# Execute the query to create the prompt
con.execute(get_embedding_prompt_query)


In [ ]:
query2 = """
WITH review_embeddings AS (
    SELECT 
        ID AS review_id,
        Review AS review_text,
        llm_complete('get-embedding', 'default', {'review': review_text}) AS embedding
    FROM product_reviews
),
review_similarities AS (
    SELECT
        r.review_id AS query_review_id,
        e.review_id AS similar_review_id,
        1 - (SQRT(SUM(
            POW(CAST(SPLIT_PART(SPLIT_PART(r.embedding, ',', 1), '[', 2) AS FLOAT) - CAST(SPLIT_PART(SPLIT_PART(e.embedding, ',', 1), '[', 2) AS FLOAT), 2) +
            POW(CAST(SPLIT_PART(r.embedding, ',', 2) AS FLOAT) - CAST(SPLIT_PART(e.embedding, ',', 2) AS FLOAT), 2) +
            POW(CAST(SPLIT_PART(SPLIT_PART(r.embedding, ',', 3), ']', 1) AS FLOAT) - CAST(SPLIT_PART(SPLIT_PART(e.embedding, ',', 3), ']', 1) AS FLOAT), 2)
        ) / 3)) AS similarity
    FROM review_embeddings r
    JOIN review_embeddings e ON r.review_id <> e.review_id
    GROUP BY r.review_id, e.review_id
)

SELECT * 
FROM review_similarities;

"""
results = con.execute(query2).fetchall()

for row in results:
    print (row)


We can now see the `top 5 similar` reviews for each review.

#### Filtering for High-Impact Reviews
Not all reviews are equally important. Some are more detailed, more critical, or offer particularly valuable feedback. In this next step, we filter out the "high-impact" reviews—those that deserve immediate attention.

This query introduces `llm_filter`, which uses the current model to determine whether a review is "high-impact" based on the sentiment, rating, and length of the review. High-impact reviews could include:

Negative reviews that are detailed and provide insights into product issues.
Positive reviews that highlight key features or advantages (useful for marketing).
By focusing on these high-impact reviews, you can address critical feedback and leverage valuable testimonials effectively.

We define a new prompt `is-high-impact-review` which classifies whether a review is 


In [ ]:
# Define the is-high-impact-review prompt
is_high_impact_review_prompt = """
Determine if the given review is a high-impact review that provides valuable insights. Consider the following factors:

1. Sentiment: {sentiment}
2. Star Rating: {rating}
3. Review Length: {review_length}

A high-impact review typically has:

- A strong sentiment (very positive or very negative)
- An extreme rating (1-2 or 4-5 stars)
- Sufficient length to provide detailed feedback (usually more than 50 words)

Output your decision as a boolean true or false.
"""

# Create the SQL query to register the is-high-impact-review prompt
is_high_impact_review_prompt_query = f"""
    CREATE PROMPT ('is-high-impact-review', '{is_high_impact_review_prompt}');
"""

# Execute the query to create the prompt
con.execute(is_high_impact_review_prompt_query)


In [ ]:
query = """
CREATE TABLE filtered_reviews AS
WITH filtered_reviews AS (
    SELECT 
        * 
    FROM 
        sentiment_analysis 
    WHERE 
        llm_filter('is-high-impact-review', 'gpt-4o', {
            'sentiment': sentiment_json, 
            'rating': star_rating, 
            'review_length': LENGTH(review_text)
        })
)
SELECT * 
FROM filtered_reviews;
"""

# Run the query and fetch results
results = con.execute(query).fetchall()

for row in results:
    print(row)

#### Extracting Key Themes from Reviews

Once you’ve identified high-impact reviews, the next step is understanding why customers leave them.
This query helps identify recurring themes such as complaints about battery life, praises for design, or issues with customer service.
This query extracts common themes using LLM's ability to understand and categorize text. You’ll get insights like:

Common Complaints: "The battery drains too quickly."
Positive Feedback: "The screen resolution is excellent."

We first define our own prompt called `extract-themes`, and then use it to extract the themes from `filtered_reviews`

In [ ]:
# Define the extract themes prompt
extract_themes_prompt = """
    Analyze the following sentiment analysis JSON. Identify and extract key themes or topics discussed in the product review. Output the themes in a JSON array format.
    
    Sentiment Analysis JSON: {{sentiment_json}}
    
    Output your response in the following JSON format:
    {{
        "themes": ["theme1", "theme2", "theme3"]
    }}
"""
       
# Create the SQL query to register the prompt
extract_themes_prompt_query = f"""
    CREATE PROMPT ('extract-themes' , '{extract_themes_prompt}');
"""

# Execute the query to create the prompt
results = con.execute(extract_themes_prompt_query)


In [ ]:
query = """
CREATE TABLE themes_extracted AS
WITH themes_extracted AS (
    SELECT 
        product_id, 
        review_id, 
        review_text,
        star_rating,
        llm_complete_json('extract-themes', 'gpt-4o', {'sentiment_json': sentiment_json}) AS themes
    FROM 
        filtered_reviews
)
SELECT * 
FROM themes_extracted;
"""
# Run the query and fetch results
results = con.execute(query).fetchall()

for row in results:
    print(row)

#### Tracking Customer Sentiment Over Time
Customer feedback changes as new product updates are rolled out or customer expectations evolve. To track customer satisfaction over time, you can analyze how the average sentiment and star ratings change month by month.

We use 2 different prompts here `sentiment_analysis_prompt` and `extract_sentiment_score_prompt_numeric`

In [ ]:
# Define the extract sentiment score prompt
extract_sentiment_score_prompt_numeric = """
Analyze the following sentiment analysis JSON and extract the sentiment score. 
Output the sentiment score as a numeric value.

Sentiment Analysis JSON: {sentiment_json}

Output your response as a number {sentiment_score}
"""

# Define the SQL query to create the prompt
query = f"""
CREATE PROMPT ('extract-sentiment-score', '{extract_sentiment_score_prompt_numeric}');
"""

# Execute the query to create the prompt
con.execute(query)


In [ ]:
# Define the SQL query
query = """
WITH sentiment_analysis_dated AS (
    SELECT 
        ProductID as product_id, 
        ID AS review_id,
        Review AS review_text, 
        Rating AS star_rating, 
        Date as review_date,
        llm_complete_json('sentiment-analysis', 'gpt-4o', {'review': review_text, 'rating': star_rating}) AS sentiment_json
    FROM 
        product_reviews
)
SELECT 
    product_id, 
    EXTRACT(YEAR FROM review_date) AS year, 
    EXTRACT(MONTH FROM review_date) AS month, 
    AVG(star_rating) AS avg_star_rating, 
    AVG(
        CAST(
            (llm_complete_json('extract-sentiment-score', 'gpt-4o', {'sentiment_json': sentiment_json}))->>'sentiment_score' AS DOUBLE
        )
    )
     AS avg_sentiment_score
FROM 
    sentiment_analysis_dated
GROUP BY 
    product_id, 
    EXTRACT(YEAR FROM review_date), 
    EXTRACT(MONTH FROM review_date)
ORDER BY 
    year DESC, month DESC;
"""

# Execute the query and fetch the results
results = con.execute(query).fetchall()

# Print the results
for row in results:
    print(row)


### And that is it, we have successfully created new prompts, used existing prompts and manipulated the standard table to gain analytical insights in our data, using the power of LLMs!

We finally close the DuckDB connection.

In [ ]:
# Close the DuckDB connection
con.close()
